In [69]:
from sklearn.model_selection import train_test_split, cross_validate
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, confusion_matrix

import pandas as pd
import numpy as np

In [70]:
data = pd.read_csv(r'G:\Other computers\My Laptop D14\Belgeler\CODING\Python.PROJECTS\MTM-DataMining-FlightDelay\manipulated_data\flightDelay_preprocessed_for_DC.csv')

df = data.copy()

In [71]:
df.head()

,Flight_Number_Marketing_Airline,AirTime,DepTimeBlk_Encoded,OriginAirport_Encoded,Target
0,1582,93.0,10,15,2
1,1583,93.0,4,100,1
2,1586,178.0,4,196,0
3,1587,55.0,8,15,0
4,1587,67.0,10,231,2


In [72]:
y = df['Target'].values
X = df.drop('Target', axis=1).values

In [73]:
np.set_printoptions(suppress=True)

In [74]:
X #Öznitelikler 

array([[1582.,   93.,   10.,   15.],
       [1583.,   93.,    4.,  100.],
       [1586.,  178.,    4.,  196.],
       ...,
       [5951.,  103.,    4.,  228.],
       [5951.,  102.,    4.,  228.],
       [5951.,  103.,    4.,  228.]])

In [75]:
y #Hedef değişken

array([2, 1, 0, ..., 0, 0, 0], dtype=int64)

In [76]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=23)

In [77]:
model = DecisionTreeClassifier(random_state=23)
model.fit(X_train, y_train)

DecisionTreeClassifier(random_state=23)

In [78]:
y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))

Accuracy: 0.4117523389046568

Confusion Matrix:
 [[3531 3126 3016]
 [3681 6932 3737]
 [4129 4695 5205]]


### Cross Validation

In [85]:
model = DecisionTreeClassifier(random_state=23)

# Cross-validation işlemi
cv_results = cross_validate(model, X, y, cv=5, scoring='accuracy', return_train_score=True)

# Test seti üzerindeki skorlar
test_scores = cv_results['test_score']
print("Test Seti Accuracy Skorları:", test_scores)
print('Test Seti Accuracy Ortalaması: ',round(test_scores.mean(), 2))

# Eğitim seti üzerindeki skorlar
train_scores = cv_results['train_score']
print("Eğitim Seti Accuracy Skorları:", train_scores)
print('Eğitim Seti Accuracy Ortalaması: ',round(train_scores.mean(), 2))


Test Seti Accuracy Skorları: [0.30400505 0.27312625 0.2286345  0.29685693 0.32979422]
Test Seti Accuracy Ortalaması:  0.29
Eğitim Seti Accuracy Skorları: [0.82231435 0.81407557 0.81952867 0.82719586 0.8204168 ]
Eğitim Seti Accuracy Ortalaması:  0.82


Şuana kadar görülebileceği üzere modelin test performansı oldukça düşüktür. Oysaki Train seti üzerinde hatırı sayılır bir performans sergilediği söylenebilir. Bu durumda modelin **overfitting** yani **aşırı öğrenme** durumu yaşadığı söylenebilir. Bu durumun önüne geçebilmek için, karar ağaçlarında **pruning** yani **budama** işlemi gerçekleştirilmelidir.

Modeli varsayılan parametreler ile kullanmak çok verimli bir yaklaşım sağlamamaktadır. Model optimizasyonu gerçekleştirip yeniden deneyelim.

### Decision Tree Model Tune

In [87]:
from sklearn.model_selection import GridSearchCV

# Karar ağacı modelini tanımlayalım
model = DecisionTreeClassifier(random_state=23)

# Parametre aralıklarını belirleyelim
param_grid = {
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': [None, 'sqrt', 'log2', 2, 3, 4, 5]
}

# GridSearchCV kullanarak en iyi parametreleri bulalım
grid_search = GridSearchCV(model, param_grid, cv=5, scoring='accuracy', return_train_score=True)
grid_search.fit(X, y)

KeyboardInterrupt: 